# Vector Applications - Vector in data analysis

Methods to evaluate the similarity between two variables
1. Correlation
2. Cosine similarity

These metrics reflect the linear relationship between two variables :
They are based on inner products, and inner product s a linear operation.